 <img align="right" src="files/img/PhUniMa_Logo_sw.svg">

*Phillips-Universität Marburg* <br>
*Fachbereich Physik*<br>
*Priv.-Doz. Dr. S.R. Manmana, WiSe 2020/21*

<h1><center>Übungen zur Vorlesung Computational Physics I<br><br>Blatt 8</center></h1>

---

# Lernziele dieses Übungsblattes


* Finite-Differenzen-Verfahren zur Diskretisierung zeitabhängiger Partieller Differentialgleichungen (partial differential equations, PDEs): Wärmeleitung, Diffusion und Wellenphänomene mit konstanter und ortsabhängiger Geschwindigkeit (Wellenausbreitung auf einem Seil, Tsunami).

* Diskretisierung (zeitabhängiger) Randbedingungen zur Behandlung dieser PDEs.

* Transiente Lösungen von parabolischen und hyperbolischen Differentialgleichungen, Stabilitätskriterien.

# Aufgabenmodus

Aufgabe 21 dieses Übungsblattes verfügt über ein Tutorial, das Sie am Ende des Dokumentes finden. Abhängig von Ihren Vorkenntnissen können Sie die Aufgabe entweder eigenständig bearbeiten, oder dem dazugehörigen Tutorial folgen. Das Tutorial auf diesem Notebook ist für eine Implementation in Python geschrieben.

# Studienleistung

Die Bearbeitung von Aufgabe 22 ist die vierte und letzte der vier unbenoteten Studienleistungen. Reichen Sie Ihre Bearbeitung (Quellcode und aussagekräftige Plots) bis zum 25. Februar per Email bei mir ein.

Verpacken Sie bitte Ihre Quelldateien, ggf. passende makefile und aussagekräftige, beschriftete Plots (PNG-Dateien) in einem ZIP- oder Tarball-Archiv, um Ihre Ordner/Projektstruktur zu erhalten. Ihnen wird die Bearbeitung testiert, wenn die Aufgabe hinreichend bearbeitet wurde. Dabei bleibt es Ihnen überlassen, ob Sie den Code in Python oder in C/C++ implementieren. Wenn Sie Python benutzen wollen, empfehle ich Ihnen, ein Jupyter-Notebook zu erstellen und dieses einzureichen. Ggf. können Sie die begleitenden Diskussion auch im Latex-Stil im Notebook aufarbeiten. Alternativ können Sie ein begleitendes pdf-Dokument mit weiterführenden Diskussionen mit einreichen, falls Sie nicht die gesamten Diskussionen im Notebook schreiben wollen.

# Übungsaufgaben


Auf diesem Übungsblatt werden Sie sich mit den Grundlagen des numerischen Lösens von partiellen Differentialgleichungen (PDEs) beschäftigen, die sowohl eine Ableitung in den Orts-, als auch der Zeitvariablen enthalten.

Während für ODEs viele Integratoren existieren, die auf eine breite Klasse von Systemen anwendbar sind, gestaltet sich die Integration von zeitabhängigen PDEs als schwieriger. In der Regel gibt es ein enges Zusammenspiel zwischen betrachtetem System und den gewählten Diskretisierungen von Raum und Zeit. Neben der Ihnen bereits bekannten Konvergenzordnung tritt vor allem auch die *Stabilität* in den Vordergrund. Die falsche Wahl des Integrators kann zu einem explosivem, exponentiellen Wachstum der Feldgrößen in der Simulation führen, wohingegen die Wahl eines *geeigneten* Integrators nur mit detaillierten mathematischen Analysen erschlossen werden kann.

## Aufgabe 21: *Wärmeleitungsgleichung (mit Tutorial)*

Die Wärmeleitungsgleichung ist der Prototyp sogenannter *parabolischer* Differentialgleichungen. Das Temperaturfeld $T(t,x)$ genügt der Bewegungsleichung

$$\frac{\partial T}{\partial t} = D\frac{\partial^2 T}{\partial x^2}$$

die die zeitliche Ableitung der Temperatur auf der linken Seite mit der zweiten räumlichen Ableitung auf der rechten Seite in Beziehung setzt. Der Parameter $D$ ist die Wärmediffusivität. Man nennt die Gleichung parabolisch, weil sie in den “Potenzen” ähnlich einer Parabelgleichung aussieht:

$$y = x^2$$

(Im Vergleich dazu: PDEs wie die Laplace-Gleichung von Blatt 7 – hier in 2D – der Form $\frac{\partial^2 \phi}{\partial x^2} + \frac{\partial^2 \phi}{\partial y^2} = 0$ werden ”elliptische PDE” genannt). Sie sollen die Wärmeleitungsgleichung in einem Stab der Länge $L$ lösen. An den Rändern des Stabes werden feste Temperaturen aufgeprägt, Sie sollen bestimmen, wie sich die Temperaturverteilung auf dem Stab mit der Zeit ändert.

<img src="files/img/Aufgabe21-1.png" style="width: 45vw;">

Nach Diskretisierung der Ortsvariablen gibt es $N$ Stützstellen mit Abstand $\Delta x$ und jeweils einer Temperatur $T_i$. Die PDE an jeder dieser Stützstellen ist somit:

$$\frac{\partial T_i}{\partial t} = D\frac{\partial^2 T_i}{\partial x^2}$$

Wie üblich bei Finite-Differenzen-Verfahren wird nun zunächst die Ableitung der Ortskoordinaten diskretisiert. Nähert man die Raumableitung durch den Ausdruck für die zentrale Differenz, so ergibt sich folgende Bewegungsgleichung:

$$\frac{\partial T_i}{\partial t} = D \frac{T_{i-1}-2T_i+T_{i+1}}{\Delta x^2}$$

Häufig werden die benachbarten Gitterpunkte mit “Ost” oder “West” (in 2D Feldern auch “Nord” und “Süd”) abgekürzt. Die drei Punkte $T_i$, $T_W$ und $T_O$ werden zusammen als 3-Punkt-Stempel (3 point stencil) bezeichnet, die PDE liest sich dann entsprechend (siehe Skizze unten):

$$\frac{\partial T_i}{\partial t} = D \frac{T_W-2T_i+T_O}{\Delta x^2}$$

<img src="files/img/Aufgabe21-2.png" style="width: 45vw;">

Die Zeitableitung auf der linken Seite der Gleichungen kann durch eine Vorwärtsdifferenz genähert werden. Das geschieht wie beim expliziten Euler-Verfahren für ODEs, indem die rechte Seite der Gleichung zum gegenwärtigen Zeitpunkt ausgewertet wird:

$$\frac{T_i(t+\Delta t) - T_i(t)}{\Delta t} = D \frac{T_W(t) - 2T_i(t) + T_O(t)}{\Delta x^2}$$

Löst man nun noch nach dem Zustand der Temperatur zum nächsten Zeitpunkt auf, ergibt sich eine Iterationsvorschrift, die zur Integration dieser (und ggf. anderer parabolischer) zeitabhängigen PDE genutzt werden kann:

$$T_i(t+\Delta t) = T_i(t) + \Delta t \cdot D \frac{T_W(t) - 2T_i(t) + T_O(t)}{\Delta x^2}$$

Diese Art die Wärmeleitungsgleichung zu diskretisieren wird häufig FTCS-Verfahren (Forward Time, Centered Space) genannt. Wie eingangs erwähnt, müssen Verfahren für PDEs individuell analysiert werden. Eine genauere Betrachtung des FTCS-Schemas für die Wärmeleitungsgleichung ergibt, dass die Wahl des Zeitschrittes $\Delta t$ bei einer gewählten räumlichen Schrittweite $\Delta x$ eingeschränkt ist, sonst erhält man unsinnige Lösungen und das Verfahren wird instabil. Man findet, dass das Verfahren genau dann *Neumann-stabil* ist, wenn folgende Ungleichung erfüllt ist:

$$\Delta t < \frac{\Delta x^2}{2D}$$

Das bedeutet, dass der Zeitschritt immer hinreichend klein gewählt werden muss, anderenfalls werden durch die Diskretisierungsfehler kleine Störungen exponentiell verstärkt.

Wie bereits auf Blatt 7 behandelt, ist es nun noch notwendig, die Randbedingungen festzulegen und bei der Simulation zu berücksichtigen. Für den FTCS-Schritt einer Stützstelle sind jeweils der Ost- oder West-Nachbar notwendig. Jedoch ist z.B. am linken Rand kein West-Nachbar verfügbar. Stattdessen werden “virtuelle” Nachbarn (manchmal “Ghosts” genannt) aus Randbedingungen generiert. Die Wahl (und richtige Implementation) der Randbedingungen kann entscheidend für die erhaltene Lösung sein.

Die zwei gängigsten Randbedingungen sind vom Dirichlet- und vom Neumann-Typ. Bei Dirichlet-Randbedingungen wird außerhalb der Simulationsdomäne ein fester Wert für die dynamische Größe festgelegt. Der virtuelle Nachbar wird also einfach auf einen Wert gesetzt; allerdings nimmt er am FTCS-Verfahren teil und der gesetzte Wert kann daher auch explizit zeitabhängig sein, d.h., man kann eine zeitabhängige Funktion für diesen Wert vorgeben:

$$Dirichlet\;  Linker\;  Rand: T_W = T_{-1} = T_{Rand}(t)$$

Neumann-Randbedingungen erfordern, dass die räumliche Ableitung der dynamischen Größe einen gewissen Wert annimmt. Damit kann z.B. ein konstanter Wärmefluss am Rand des Systems erzwungen werden.

Beispiel: die Ableitung der Temperatur am rechten Rand soll den Wert $w$ haben,

$$\left. \frac{\partial T_N}{\partial x}\middle|_{rechterHand} \right.= w$$

Mit Hilfe der finiten Differenz (Stirling-Ableitung) wird daraus

$$\frac{T_O - T_W}{2\cdot \Delta x} = \frac{T_{N+1} - T_{N-1}}{2\cdot \Delta x} = w,$$

wobei $T_O$ nun ein virtueller Nachbar sein muss, da wir am rechten Rand sind. Aufgelöst nach diesem Ghost $T_O$ ergibt sich

$$T_O = T_W + 2w\Delta x.$$

Wenn der Wärmefluss am rechten Rand also vollständig unterdrückt werden soll (Isolation mit $w = 0$), so gilt

$$Neumann\; Rechter\;  Rand\;  (kein\;  Fluss\;  durch\;  den\;  Rand): T_O = T_{N+1} = T_W$$

**Hinweis**: Die Aufgabe verfügt über ein Tutorial.

1. Schreiben Sie die Funktion

In [1]:
def heatFTCS(t, y, dt):
    pass

die für 1D-Wärmeleitungsgleichung einen einzelnen FTCS-Schritt durchführt. Die Domäne hat eine Länge von $L = 1$ und wird mit $\Delta x = 0.01$ diskretisiert. Es gelte $D = 0.1$.

Die Funktion nimmt einen Zustandsvektor $y$ entgegen, der die $N$ Temperaturen der Domäne enthält. Berechnen Sie die neuen Werte des Feldes und schreiben Sie sie zurück in $y$. Achten Sie darauf, dass Sie die Ergebnisse zunächst in ein Zwischenarray schreiben müssen, damit die rechte Seite der PDE immer zum Zeitpunkt $t$ ausgewertet wird!

2. Initialisieren Sie das Feld auf die konstante Temperatur $T = 0$. Prägen Sie dem System am linken Rand die konstante Temperatur $T_{links} = 1$ und am rechten Rand die konstante Temperatur $T_{rechts} = -1$ auf.

3. Simulieren Sie das System für einige Zeit und stellen Sie die zeitliche Entwicklung des Temperaturfeldes geeignet dar (z.B. mithilfe einer Animation in einem Jupyter-Notebook oder Python-Skript). Verwenden Sie einmal einen Zeitschritt unterhalb des Stabilitätskriteriums und einmal oberhalb des Stabilitätskriteriums. Verändern Sie auch einige Parameter des Systems und überzeugen Sie sich davon, dass das Kriterium tatsächlich funktioniert.

4. Setzen Sie den rechten Rand auf einen Neumann-Rand mit $w = 0$. Erzeugen Sie am linken Rand des Systems mit einem zeitabhängigen Dirichlet-Rand einen Wellenberg:

$$T_{links}(t) = \exp(-(t-5)^2)$$

Stellen Sie ebenfalls den zeitlichen Verlauf des Wellenbergs dar.

5. **Optional 1**: Bestimmen Sie numerisch die ”Zerfallskonstante” der Amplitude und die Ausbreitungsgeschwindigkeit des Wellenbergs.

6. **Optional 2**: Setzen Sie beide Enden der Domäne auf Neumann-Ränder. Pumpen Sie am linken Rand für eine Zeitdauer von 1s Wärme in das System und isolieren Sie danach das System. Wie gut wird nach dem Ende der Wärmezufuhr die Gesamtenergie im System erhalten?

### Aufgabenlösung 21

## Aufgabe 22: *Diffusionsgleichung (Studienleistung)*

<img src="files/img/Aufgabe22.png" style="width: 55vw;">

Ein der Wärmeleitung mathematisch äquivalenter physikalischer Prozess ist die Diffusion. Wenn Sie einen Tropfen Farbe vorsichtig in eine Flüssigkeit geben ohne diese in Bewegung zu setzen, so verteilt sich die Farbe mit der Zeit in der ganzen Flüssigkeit. Die Konzentration $c(x, t)$ des Farbstoffs folgt der Gleichung

$$\frac{\partial c}{\partial t} = D\frac{\partial^2 c}{\partial x^2}$$

Hierbei ist $D$ der Diffusionskoeffizient.

Sie können diese Gleichung auch als eine Kontinuitätsgleichung

$$\frac{\partial c}{\partial t} + \frac{\partial j}{\partial x} = 0$$

mit der Flussdichte

$$j = -D\frac{\partial c}{\partial x} \tag{1}$$

auffassen, also einem Fluss des Farbstoffs immer gegen den Gradienten der Konzentration: Der Farbstoff diffundiert in die Richtung, in die die Konzentration abnimmt.

Die Flüssigkeit befinde sich in einem geschlossenen Gefäß. Daher kann es keinen Fluss des Farbstoffes durch die Gefäßwände geben, d.h., es gilt wegen Gl. (1) die Neumann-Randbedingung

$$j = 0 \longrightarrow \frac{\partial c}{\partial x} = 0$$

an beiden Rändern. Betrachten Sie einen zur Zeit $t = 0$ in der Mitte des Gefäßes lokalisierten Tropfen des Farbstoffes und integrieren Sie die Diffusionsgleichung, um seine Ausbreitung zu untersuchen.

1. Plotten Sie das Dichteprofil zu verschiedenen Zeitpunkten, bis der Farbstoff gleichmäßig im Gefäß verteilt ist.

    *Hinweis*: Besonders für einen scharfen Peak (also einen einzelnen, von Null verschiedenen Gitterpunkt) als Startbedingung für die Konzentration müssen Sie ggfs. $\Delta t$ hinreichend weit vom Stabilitätspunkt entfernen, um auf eine physikalisch plausible Lösung zu kommen.

2. Solange der Farbstoff die Gefäßwände noch nicht erreicht hat, kann sein Dichteprofil näherungsweise (abgesehen von Diskretisierungseffekten) durch eine Gauß-Kurve beschrieben werden. Bestimmen Sie die Breite dieser Gaußkurve und plotten Sie diese als Funktion der Zeit.

    **Tipp**: Die “Breite” der Kurve kann auf viele verschiedene Arten bestimmt werden. Sie können z.B. die Halbwertsbreite bestimmen, also den Abstand der beiden Gitterpunkte, an denen die Konzentration auf die Hälfte der maximalen Konzentration abgefallen ist. Sie können auch in Python eine Gaußkurve an die Konzentrationskurve anfitten und daraus die Breite extrahieren. Eine dritte Möglichkeit wäre, die Standardabweichung des Konzentrationsprofils zu bestimmen.

Falls Sie das Konzentrationsprofil an das Profil der Aufenthaltswahrscheinlichkeit eines Random Walkers erinnert, liegen Sie richtig: Sie können sich die diffusive Ausbreitung des Farbtropfens als das Ergebnis der Zufallsbewegungen der Farbstoffmoleküle vorstellen.

Abzugeben: Programm-Code und Plots der verschiedenen Konzentrationsprofile und der Zeitabhängigkeit der Gauß-Breite.

### Aufgabenlösung 22

## Aufgabe 23: *Wellenphänomene (Bonus)*

In der Vorlesung haben wir die d’Alembert-Gleichung

$$\frac{\partial^2 f(\vec{x},t)}{\partial t^2} = u^2 \nabla^2 f(\vec{x}, t) \tag{2}$$

betrachtet, die auch ”Wellengleichung” genannt wird, da sie die Ausbreitung von Wellen in drei Raumdimensionen beschreibt (wegen ihrer Form wird sie als eine ”hyperbolische” PDE bezeichnet).

Wir betrachten nun die Ausbreitung von Wellen in einem eindimensionalen System. Im allgemeinen Fall gehen wir aber davon aus, dass die Wellengeschwindigkeit von der Position abhängt, was wir durch eine verallgemeinerte d’Alembert-Gleichung

$$\frac{\partial^2 f}{\partial t^2} = \frac{\partial}{\partial x}
\left( u^2(x)\frac{\partial f}{\partial x} \right) = u^2(x)
\frac{\partial^2 f(x)}{\partial x^2} + \frac{\partial u^2(x)}{\partial x}\frac{\partial f(x)}{\partial x} \tag{3}$$

beschreiben. Für $u(x) = const.$ ergibt sich dabei wieder die in der Vorlesung behandelte d’Alembertgleichung in 1D. Zur Behandlung einer ortsabhängigen Wellengeschwindigkeit muss daher im Vergleich zum in der Vorlesung behandelten Schema noch zusätstzlich der 2. Term auf der rechten Seite von Gl. (3) behandelt werden. Dies geschieht durch Diskretisierung der beiden ersten Ableitungen mittels der in der Vorlesung beschriebenen Stirling-Ableitung

$$f(x)' \approx \frac{f(x_{i+1}) - f(x_{i-1})}{2\Delta x}$$

und man erhält für das finite Differenzen-Verfahren den Ausdruck

$$f(x_i, t_{n+1}) = 2(1-2\beta^2_i)f(x_i,t_n) - f(x_i,t_{n-1}) + \beta^2_i(f(x_{i+1},t_n) + f(x_{i-1},t_n)) + \frac{1}{4} (\beta^2_{i+1} - \beta^2_{i-1}) (f(x_{i+1},t_n) - f(x_{i-1},t_n))$$

Dabei ist $\beta_i = u(x_i)\Delta t/\Delta x$ der nun ortsabhängige CFL-Parameter.

1. Vergewissern Sie sich, dass die Diskretisierung der verallgemeinertend ’Alembertgleichung(3) den Ausdruck (4) ergibt. Was ergibt sich für $u(x) = const.$?

2. Zum Testen eines Algorithmus ist es immer sinnvoll, diesen auf eine Situation anzuwenden, bei der man die Lösung bereits kennt. Betrachten Sie dazu ein an einer Wand befestigtes Seil, dessen freies Ende periodisch mit einer Funktion

$$A(t) = A_0 sin(\omega t)$$

angetrieben wird. Was erwarten Sie, und was beobachten man mit Variation von $\omega$? Stimmen die Ergebnisse mit Ihrer Erwartung überein?

*Hinweise*: (i) Implementieren Sie das finite Differenzen-Schema mit den entsprechenden Randbedingungen in ein von Grund auf selbst geschriebenem Programm. Achten Sie auf Stabilität (d.h., dass der CFL-Parameter $\beta < 1$). (ii) Gehen Sie von $u(x) = const.$ aus. (iii) Sie können probeweise auch Rechnungen mit CFL- Parameter $\beta > 1$ probieren, um zu sehen, was passiert, wenn das Stabilitätskriterium nicht beachtet wird.

3. Tsunami: Wir betrachten nun in einer vereinfachten Weise die Situation, wie sie sich an Ozeanen angrenzenden Küsten nach einem Seebeben realisieren kann. Wir betrachten dabei eine Welle, die sich über ein Korrallenriff (z.B. dem Great-Barrier-Riff vor Australien) hinwegbewegen soll. Das Riff hat dabei an einigen Stellen ein Plateau, das ggf. über das Wasser ragen und dort einen flachen Sandstrand bilden kann. Wir gehen nun davon aus, dass das Seebeben in einiger Entfernung zum Riff in einigen Kilometern Wassertiefe eine sinusartige Anregung der Wasseroberfläche mit Amplitude $A_0$ verursacht, und dass diese Anregung nur eine Periode andauert. Die Geschwindigkeit der Wasserwellen hängt nun von der Meerestiefe wie $u(x) = \sqrt{gh(x)}$ ab. Der Meeresboden habe dabei ein Profil

$$h(x) = 
\begin{cases} 
      h_{Ozean}, & x \in [x_L, x_{Riff,L}]\\
      h_{Ozean} + (h_{Riff} - h_{Ozean}) sin^2\left(\frac{\pi(x-x_{Riff,L})}{2(x_{Strand,L} - x_{Riff,L})}\right), & x \in [x_{Riff,L}, x_{Strand,L}]\\
      h_{Riff}, & x \in [x_{Strand,L}, x_{Strand,R}]\\
      h_{Riff} - (h_{Riff} - h_{Ozean}) sin^2\left(\frac{\pi(x_{Strand,R}-x)}{2(x_{Strand,R} - x_{Riff,R})}\right), & x \in [x_{Strand,R}, x_{Riff,R}]\\
      h_{Ozean}, & x \in [x_{Riff,R}, x_R]
\end{cases}$$

<img src="files/img/Aufgabe23.png" style="width: 45vw;">

s. Abbildung. Dabei sei $$h_{Ozean} = 5000m\\ h_{Riff} = 20m\\ x_L = -1000km\\ x_R = 0\\ x_{Riff,L} = -700km\\ x_{Strand,L} = -550km\\ x_{Strand,R} = -450km$$ und $$x_{Riff,R} = -300km$$. Die Störung zu Beginn der Simulation sei zunächst null $(f(x,0) = 0, \forall x)$ und kann dann als zeitabhängige Randbedingung am linken Rand aufgefasst werden,
    
$$f(x_l,t) = 
\begin{cases} 
      A_0 sin(\omega t), & 0 \leqslant t \leqslant 2\pi/\omega\\
      0, & t > 2\pi/\omega
\end{cases}$$
    
mit einer Periode $T = 2\pi/\omega = 2000s$. Wählen Sie $\Delta t$ so dass $\beta_{CFL} \leqslant 1 \forall t, \forall x$. Wählen Sie verschiedene Diskretisierungen der Raumkoordinate, aber immer so, dass mindestens 2 Gitterpunkte pro Wellenlänge vorhanden sind und bestimmen Sie, wann die Ergebnisse konvergieren. Führen Sie die Simulationen bis $t_{max} = 22000s$ durch. Bestimmen Sie die Wasserhöhe als Funktion des Ortes und der Zeit mit Hilfe des oben angegebenen finite Differenzen-Verfahrens für verschiedene Werte von $A_0 \in [0.5m,10ms]$. Was sehen Sie bei Verkleinerung von $h_{Riff}$, und wie klein können Sie den Wert noch wählen, so dass die Simulation noch stabil läuft?


*Hinweise*: (i) Gehen Sie von einem eindimensionalen System aus und vernachlässigen Sie alle weiteren Einflüsse wie Meeresströmungen, Wind, etc. (ii) Behalten Sie möglicherweise singuläres Verhalten im Auge und brechen Sie wenn nötig die Rechnung rechtzeitig ab.

### Aufgabenlösung 23

# Selbsttest

* Was ist das FTCS-Verfahren?
* Wie kann man sich Dirichlet- und Neumann-Randbedingungen veranschaulichen?
* Was sind die Stabilitätskriterien für die auf diesem Blatt beschriebenen Algorithmen? Was passiert, wenn man diese Kriterien nicht einhält?
* Was versteht man unter Finite-Differenzen-Verfahren? Kann man diese in beliebigen Dimensionen formulieren?
* Inwiefern kann man die Diffusion eines Farbtropfens in einer Flüssigkeit durch einen Strom beschreiben?

# Tutorials

## Tutorial 21: Wärmegleichung

Dieses Tutorial ist für eine Implementation in Python geschrieben.

1. Legen Sie in `heatFTCS` zunächst ein neues Array für den nächsten Zeitschritt an.

In [36]:
y_new = np.zeros(N)

2. Beginnen Sie mit einer `for`-Schleife über sämtliche Gitterpunkte:

In [25]:
for i in range(N):
    pass

3. Erstellen Sie in der Schleife drei Hilfsvariablen mit den Werten für $T_W$, $T_O$ und $T_i$:

In [ ]:
T = y[i % N]
TW = y[(i-1) % N]
TO = y[(i+1) % N]

Die Modulo-Operation stellt sicher, dass die Zugriffe auf das Array innerhalb der Speichergrenzen geschieht. Dadurch werden automatisch periodische Randbedingungen erzeugt.

4. Implementieren Sie die Ränder, die nicht periodisch sein sollen. Eine Neumann-Bedingung am rechten Rand kann z.B. wie folgt erstellt werden:

In [ ]:
if (i == N-1):
    TO = TW

5. Berechnen Sie nun den neuen Wert des Gitterpunktes gemäß der Iterationsvorschrift

In [ ]:
y_new[i] = y[i] + dt * ...

6. Nachdem Sie alle Gitterpunkte berechnet haben, übertragen Sie den Inhalt von `y_old` nach `y`.

7. Erstellen Sie den Rest der Simulation. Geben Sie den Zustand des Feldes zeilenweise pro Zeitschritt in eine Datei aus.

8. Sie können die entstehende “2D-Grafik” in Python z.B. mit `imshow` visualisieren:

In [ ]:
data = np.loadtxt("heat.dat", delimiter=",")
plt.imshow(data, aspect=’auto’)
plt.show()

Wenn Sie die einzelnen Temperaturkurven lieber als Graphen sehen wollen, können Sie mit `.T` ein Array transponieren. Mit folgendem Befehl plotten Sie halbtransparent jede zehnte Zeile Ihrer Ausgabe:

In [ ]:
plt.plot(data[::10,:].T, alpha = 0.5)